In [1]:
import cv2
import json
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import numpy as np

In [2]:
def get_frames_set(ball_json, event_json):
    #open ball positions
    with open(ball_json, 'r') as f:
        ball_data = json.load(f)
    #open event positions and transform classes to model output value
    with open(event_json, 'r') as f:
        events = {"bounce": np.array([1,0,0]), "net": np.array([0,1,0]), "empty_event": np.array([0,0,1])}
        event_data = json.load(f)
        event_data = {frame: events[event] for frame, event in event_data.items()}
    #find intersection b/w keys of ball and event data 
    #commonality acts as the training set
    train_frames = set(event_data.keys()) & set(ball_data.keys())
    ball_data = {frame: ball_data[frame] for frame in train_frames}
    event_data = {frame: event_data[frame] for frame in train_frames}
    print(f"Train frames: {len(train_frames)}")
    print(f"Ball data: {len(ball_data)}")
    print(f"Event data: {len(event_data)}")
    return train_frames, ball_data, event_data



In [3]:
def crop_centered_with_padding(image, center, crop_size):
    img_h, img_w = image.shape[:2]
    crop_w, crop_h = crop_size
    cx, cy = center

    # Calculate crop box coordinates
    x1 = int(cx - crop_w // 2)
    y1 = int(cy - crop_h // 2)
    x2 = x1 + crop_w
    y2 = y1 + crop_h

    # Determine padding if crop box is outside image boundaries
    pad_left = max(0, -x1)
    pad_top = max(0, -y1)
    pad_right = max(0, x2 - img_w)
    pad_bottom = max(0, y2 - img_h)

    # Adjust crop box to fit inside image boundaries
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(img_w, x2)
    y2 = min(img_h, y2)

    # Crop the valid region
    cropped = image[y1:y2, x1:x2]

    # Pad to maintain the same size
    cropped_padded = cv2.copyMakeBorder(
        cropped,
        pad_top, pad_bottom,
        pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]  # black padding
    )

    return cropped_padded


In [4]:
def extract_specific_frames(video_path, frame_indices):
    cap = cv2.VideoCapture(video_path)
    extracted_frames = {}

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return extracted_frames

    for index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        success, frame = cap.read()
        if success:
            extracted_frames[index] = frame
        else:
            print(f"Warning: Could not read frame {index}")

    cap.release()
    return extracted_frames


In [5]:
def data_preprocess(video_path, ball_json, event_json):
    # Step 1: Get frames set
    train_frames, ball_data, event_data = get_frames_set(ball_json, event_json)
    
    train_frames = list(map(int, event_data.keys()))
    print("HERE ARE THE TRAIN FRAMES")
    #print(train_frames)
    frames = extract_specific_frames(video_path, train_frames)
    # Step 3: Center crop frames
    cropped_frames = []
    for frame in frames:
        frame_num = str(frame)
        center = (int(ball_data[frame_num]['x']), int(ball_data[frame_num]['y']))
        cropped_frame = crop_centered_with_padding(frames[frame], center, (320, 220))
        cropped_frames.append(cropped_frame)
    print("Frames all cropped around ball coordinate")
    return cropped_frames, ball_data, event_data

In [9]:
model_path = "../models/ball_event_model.keras"
model = tf.keras.models.load_model(model_path, compile = False)
lr_scheduler = ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=100,
        decay_rate=0.96)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
input_shape=(220, 320, 3)
inputs = layers.Input(shape=input_shape)
outputs = model(inputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics = ['accuracy', "f1_score"])

In [ ]:
test_folder = "../data/test/"
accuracies = []
labels = []
for i in range(1,8):
    if i !=4:
        video_file = test_folder + f"test_{i}.mp4"
        ball_markup = test_folder + f"test_{i}/ball_markup.json"
        events_markup = test_folder +f"test_{i}/events_markup.json"
        cropped_frames, ball_data, events_data = data_preprocess(video_file, ball_markup, events_markup)
        # print(cropped_frames)
        event_data = []
        for event in events_data.values():
            event_data.append(event)
        cropped_frames = np.array(cropped_frames)
        outputs = np.array(event_data)
        results = model.predict(cropped_frames)
        arg_out = tf.argmax(outputs, axis =1)
        
        y_pred_classes = tf.argmax(results, axis=1)

        # Compute accuracy
        accuracy = tf.reduce_mean(tf.cast(tf.equal(arg_out, y_pred_classes), tf.float32))
        accuracies.append(accuracy)
        labels.append(f"Video {i}")
        print("Accuracy:", accuracy.numpy())

Train frames: 63
Ball data: 63
Event data: 63
HERE ARE THE TRAIN FRAMES
Frames all cropped around ball coordinate
2/2 ━━━━━━━━━━━━━━━━━━━━ 68s 31s/step
Accuracy: 0.41269842
Train frames: 80
Ball data: 80
Event data: 80
HERE ARE THE TRAIN FRAMES
Frames all cropped around ball coordinate
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 25s/step
Accuracy: 0.475
Train frames: 55
Ball data: 55
Event data: 55
HERE ARE THE TRAIN FRAMES
Frames all cropped around ball coordinate
2/2 ━━━━━━━━━━━━━━━━━━━━ 56s 24s/step
Accuracy: 0.43636364


In [ ]:
import matplotlib.pyplot as plt

# Create the bar chart
plt.figure(figsize=(10, 6))
plt.bar(labels, accuracies, color='skyblue')

# Add title and labels
plt.title('Ball Events Accuracy')
plt.xlabel('Video Number')
plt.ylabel('Accuracy')

# Optionally add grid and display
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()
